<a href="https://colab.research.google.com/github/kmalhotra18/HuggingFace/blob/main/Meeting_Minutes_Product.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Model to **Create Meeting Minutes**


1.   Take audio recording
2.   Use frontier model to then covert audio to text
3.   Use open source model to generate minutes
4.   Stream back results in markdown



In [ ]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai httpx==0.27.2

In [ ]:
!pip install gradio

In [ ]:
# imports

import os
import torch
import gradio as gr
from openai import OpenAI
from IPython.display import Markdown, display
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig

In [ ]:
# Constants

AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [ ]:
# New capability - connect this Colab to my Google Drive
# See immediately below this for instructions to obtain denver_extract.mp3

# drive.mount("/content/drive")
# New capability - connect this Colab to my Google Drive
# See immediately below this for instructions to obtain denver_extract.mp3

# drive.mount("/content/drive")
# audio_filename = "/content/drive/MyDrive/llms/denver_extract.mp3"

audio_filename = "/content/denver_extract.mp3"

In [ ]:
# Sign in to HuggingFace Hub

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [ ]:
# Sign in to OpenAI using Secrets in Colab

openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

In [ ]:
# Use the Whisper OpenAI model to convert the Audio to Text

audio_file = open(audio_filename, "rb")
transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text")
print(transcription)

In [ ]:
#Prompts for Llama3

system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
user_prompt = f"Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [ ]:
# quant_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=torch.bfloat16,
#     bnb_4bit_quant_type="nf4"
# )

In [ ]:
#Create tokenizer for Llama using AutoTokenizer. Then Apply chat function to pass in transcription into our GPU. Then stream the results.

# tokenizer = AutoTokenizer.from_pretrained(LLAMA)
# tokenizer.pad_token = tokenizer.eos_token
# inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
# streamer = TextStreamer(tokenizer)
# model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
# outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

In [ ]:
# Load tokenizer and model just once to reuse
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    LLAMA,
    device_map="auto",
    quantization_config=quant_config
)

In [ ]:
# Meeting Minutes Generation Function
def generate_minutes(audio_path):
    # Step 1: Transcribe audio with Whisper via OpenAI
    with open(audio_path, "rb") as audio_file:
        transcript = openai.audio.transcriptions.create(
            model=AUDIO_MODEL,
            file=audio_file,
            response_format="text"
        )

    # Step 2: Prepare prompt for LLaMA
    system_message = (
        "You are an assistant that produces minutes of meetings from transcripts, "
        "with summary, key discussion points, takeaways and action items with owners, in markdown."
    )

    user_prompt = (
        f"Below is an extract transcript of a Denver council meeting. "
        f"Please write minutes in markdown, including a summary with attendees, location and date; "
        f"discussion points; takeaways; and action items with owners.\n\n{transcript}"
    )

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt}
    ]

    # Step 3: Tokenize and Generate
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
    streamer = TextStreamer(tokenizer)
    outputs = model.generate(inputs, max_new_tokens=2000)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response


In [ ]:
# Gradio Interface
iface = gr.Interface(
    fn=generate_minutes,
    inputs=gr.Audio(type="filepath", label="Upload Council Meeting Audio (MP3/WAV)"),
    outputs=gr.Markdown(label="📝 Meeting Minutes"),
    title="Meeting Minutes Generator",
    description="Upload an audio recording of a meeting and receive AI-generated minutes with summaries, action items, and key points."
)

In [ ]:
# Launch Gradio app
iface.launch(share=True)

In [ ]:
#Get the decoded text by taking the output

# response = tokenizer.decode(outputs[0])

In [ ]:
# display(Markdown(response))